In [ ]:
import pandas as pd
import numpy as np

import keras
import keras_tuner
from hp_tuning__v1_multi import LSTMHypermodel
from hp_tuning__v1_multi import TRAINING_WINDOW_SIZE, PREDICTED_WINDOW_SIZE, N_FEATURES_SEQ, N_FEATURES_STC, N_SAMPLES, BATCH_SIZE

In [ ]:
PROJECT = 'try1230'

lstm_hypermodel = LSTMHypermodel(
    training_window_size=TRAINING_WINDOW_SIZE,
    predicted_window_size=PREDICTED_WINDOW_SIZE, 
    n_features_seq=N_FEATURES_SEQ, 
    n_features_stc=N_FEATURES_STC,
    samples_per_epoch=N_SAMPLES,
    batch_size=BATCH_SIZE
)

tuner = keras_tuner.BayesianOptimization(
    hypermodel=lstm_hypermodel,
    objective=keras_tuner.Objective(name='val_root_mean_squared_error', direction='min'),
    max_trials=15,
    directory='tuning',
    project_name=PROJECT,
    overwrite=False
)

tuner.reload()

In [ ]:
def retrieve_tuning_summary(tuner: keras_tuner.Tuner) -> pd.DataFrame:

    objective_name = tuner.oracle.objective.name

    results = [
        {
            **{"trial_id": trial_id}, 
            **{"objective_val": trial.metrics.get_best_value(objective_name)}, 
            **trial.hyperparameters.values
        } 
        for trial_id, trial 
        in tuner.oracle.trials.items()
    ]

    return pd.DataFrame(results).set_index('trial_id').sort_values('objective_val', ascending=True)

tuning_summary = retrieve_tuning_summary(tuner)

In [ ]:
tuning_summary